In [4]:
import pandas as pd
import numpy as np

In [5]:
df = pd.read_csv('loan.csv')


In [6]:
df.head()

,Loan_ID,loan_status,Principal,terms,effective_date,due_date,paid_off_time,past_due_days,age,education,Gender
0,xqd20166231,PAIDOFF,1000,30,9/8/2016,10/7/2016,9/14/2016 19:31,NaN,45,High School or Below,male
1,xqd20168902,PAIDOFF,1000,30,9/8/2016,10/7/2016,10/7/2016 9:00,NaN,50,Bechalor,female
2,xqd20160003,PAIDOFF,1000,30,9/8/2016,10/7/2016,9/25/2016 16:58,NaN,33,Bechalor,female
3,xqd20160004,PAIDOFF,1000,15,9/8/2016,9/22/2016,9/22/2016 20:00,NaN,27,college,male
4,xqd20160005,PAIDOFF,1000,30,9/9/2016,10/8/2016,9/23/2016 21:36,NaN,28,college,female


In [7]:
print( df.shape)


(500, 11)


In [8]:
df = df.dropna(subset=['Principal', 'terms', 'age'])


In [9]:
df['effective_date'] = pd.to_datetime(df['effective_date'])
df['due_date'] = pd.to_datetime(df['due_date'])
df['paid_off_time'] = pd.to_datetime(df['paid_off_time'])


In [10]:
print(df.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Loan_ID         500 non-null    object        
 1   loan_status     500 non-null    object        
 2   Principal       500 non-null    int64         
 3   terms           500 non-null    int64         
 4   effective_date  500 non-null    datetime64[ns]
 5   due_date        500 non-null    datetime64[ns]
 6   paid_off_time   400 non-null    datetime64[ns]
 7   past_due_days   200 non-null    float64       
 8   age             500 non-null    int64         
 9   education       500 non-null    object        
 10  Gender          500 non-null    object        
dtypes: datetime64[ns](3), float64(1), int64(3), object(4)
memory usage: 43.1+ KB
None


In [11]:
avg_principal = df['Principal'].mean()
print(f"\nAverage loan amount: {avg_principal:.2f}")


Average loan amount: 943.20


In [12]:
max_age = df['age'].max()
min_age = df['age'].min()
print(f"Oldest borrower: {max_age} years")
print(f"Youngest borrower: {min_age} years")


Oldest borrower: 51 years
Youngest borrower: 18 years


In [13]:
status_counts = df['loan_status'].value_counts()
print("\nLoan status counts:")
print(status_counts)



Loan status counts:
loan_status
PAIDOFF               300
COLLECTION            100
COLLECTION_PAIDOFF    100
Name: count, dtype: int64


In [14]:
paidoff_pct = (df['loan_status'] == 'PAIDOFF').mean() * 100
print(f"\nPercentage of paidoff loans: {paidoff_pct:.1f}%")


Percentage of paidoff loans: 60.0%


In [15]:
df['actual_duration'] = (df['paid_off_time'] - df['effective_date']).dt.days


In [16]:
shortest_loan = df.loc[df['actual_duration'].idxmin(), 'Loan_ID']
print(f"\nLoan ID with shortest duration: {shortest_loan}")



Loan ID with shortest duration: xqd20160706


In [17]:
edu_dist = df['education'].value_counts()
print("\nLoan distribution by education:")
print(edu_dist)


Loan distribution by education:
education
college                 220
High School or Below    209
Bechalor                 67
Master or Above           4
Name: count, dtype: int64


In [18]:
avg_age_by_gender = df.groupby('Gender')['age'].mean()
print("\nAverage age by gender:")
print(avg_age_by_gender)


Average age by gender:
Gender
female    31.883117
male      30.976359
Name: age, dtype: float64


In [19]:
avg_by_term = df.groupby('terms')['Principal'].mean()
print("\nAverage principal by term length:")
print(avg_by_term)



Average principal by term length:
terms
7     752.380952
15    893.719807
30    995.588235
Name: Principal, dtype: float64


In [20]:
df['status_numeric'] = df['loan_status'].map({'PAIDOFF': 1, 'COLLECTION': 0})


In [21]:
correlations = df[['Principal', 'terms', 'age', 'status_numeric']].corr()
print("\nCorrelation matrix:")
print(correlations)



Correlation matrix:
                Principal     terms       age  status_numeric
Principal        1.000000  0.534302 -0.092613       -0.068454
terms            0.534302  1.000000 -0.074259       -0.092776
age             -0.092613 -0.074259  1.000000        0.043115
status_numeric  -0.068454 -0.092776  0.043115        1.000000


In [22]:
past_due = df['past_due_days'].notna().sum()
print(f"\nNumber of past due loans: {past_due}")


Number of past due loans: 200


In [23]:
common_edu_female = df[df['Gender'] == 'female']['education'].mode()[0]
print(f"Most common education among female borrowers: {common_edu_female}")


Most common education among female borrowers: college


In [24]:
bins = [0, 25, 35, 45, 55, 100]
labels = ['18-25', '26-35', '36-45', '46-55', '56+']
df['age_group'] = pd.cut(df['age'], bins=bins, labels=labels)

In [25]:
default_rates = df.groupby('age_group')['loan_status'].apply(
    lambda x: (x == 'COLLECTION').mean() * 100)
print("\nDefault rate by age group:")
print(default_rates)


Default rate by age group:
age_group
18-25    24.000000
26-35    19.496855
36-45    19.354839
46-55    14.285714
56+            NaN
Name: loan_status, dtype: float64


C:\Users\User\AppData\Local\Temp\ipykernel_23508\2773353612.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  default_rates = df.groupby('age_group')['loan_status'].apply(
